In [31]:
import pandas as pd
import numpy as np
from pathlib import Path
from pandas.testing import assert_frame_equal
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
p1 = Path.cwd() / 'back_data'

In [2]:
flights = pd.read_csv(p1 / 'flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [3]:
# groupby() 메서드의 호출 결과는 groupby 객체 -> 메서드를 체인시킬 때 결과 출력
# groupby 객체에 agg(), filter(), transform(), apply() 메서드 활용 가능
(flights
# 그룹화 열('AIRLINE'), 집계열('ARR_DELAY' -> 리스트 전달로 DataFrame 가능), 집계함수(agg('mean')) 확인
.groupby('AIRLINE')[['ARR_DELAY']]
# agg() 메서드와 함께 집계 함수를 사용하지 않으면 예외 발생
.agg('mean')
.head()
)

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580


In [4]:
(flights
# 요일별로 모든 항공사의(그룹화 열) 취소, 우회한 항공편(집계열)
.groupby(['AIRLINE', 'WEEKDAY'])[['CANCELLED', 'DIVERTED']]
# 그 수와 비율(집계 함수) -> CANCELLED, DIVERTED 열은 1과 0으로 정리되어있기 때문
.agg(['sum', 'mean'])
.head()
)

CANCELLED           DIVERTED          
                      sum      mean      sum      mean
AIRLINE WEEKDAY                                       
AA      1              41  0.032106        6  0.004699
        2               9  0.007341        2  0.001631
        3              16  0.011949        2  0.001494
        4              20  0.015004        5  0.003751
        5              18  0.014151        1  0.000786

In [5]:
(flights
# 여러 그룹화열이 있을 경우 계층적 index 생성
.groupby(['ORG_AIR', 'DEST_AIR'])
# 여러 집계열이 있을 경우 계층적 columns 생성
# 집계열과 집계함수를 딕셔너리 형태로 정리 가능
.agg({'CANCELLED':['sum', 'mean', 'size'], 'AIR_TIME':['mean', 'var']})
.head()
)

CANCELLED              AIR_TIME           
                       sum mean size        mean        var
ORG_AIR DEST_AIR                                           
ATL     ABE              0  0.0   31   96.387097  45.778495
        ABQ              0  0.0   16  170.500000  87.866667
        ABY              0  0.0   19   28.578947   6.590643
        ACY              0  0.0    6   91.333333  11.466667
        AEX              0  0.0   40   78.725000  47.332692

In [6]:
# pd.NamedAgg() 함수를 통해 비계층 열 생성 가능 -> columns의 level이 한 단계
(flights
.groupby(['ORG_AIR', 'DEST_AIR'])
# agg() 메서드 내 {column_name}=pd.NamedAgg() 활용 -> column과 aggfunc 파라미터로 진행
.agg(sum_cancelled=pd.NamedAgg(column='CANCELLED', aggfunc='sum'),
mean_cancelled=pd.NamedAgg(column='CANCELLED', aggfunc='mean'),
# size 집계 함수는 그룹당 총 행의 개수 반환 / count 집계 함수는 그룹당 비결측치 개수를 반환
size_cancelled=pd.NamedAgg(column='CANCELLED', aggfunc='size'),
mean_air_time=pd.NamedAgg(column='AIR_TIME', aggfunc='mean'),
var_air_time=pd.NamedAgg(column='AIR_TIME', aggfunc='var'))
.head()
)

sum_cancelled  mean_cancelled  size_cancelled  \
ORG_AIR DEST_AIR                                                  
ATL     ABE                   0             0.0              31   
        ABQ                   0             0.0              16   
        ABY                   0             0.0              19   
        ACY                   0             0.0               6   
        AEX                   0             0.0              40   

                  mean_air_time  var_air_time  
ORG_AIR DEST_AIR                               
ATL     ABE           96.387097     45.778495  
        ABQ          170.500000     87.866667  
        ABY           28.578947      6.590643  
        ACY           91.333333     11.466667  
        AEX           78.725000     47.332692

In [7]:
res = (flights
.groupby(['ORG_AIR', 'DEST_AIR'])
.agg({'CANCELLED':['sum', 'mean', 'size'], 'AIR_TIME':['mean', 'var']})
.head()
)
# res의 columns를 columns 속성의 to_flat_index() 메서드 및 list comprehension을 활용하여 펼치기 가능
res.columns = ['_'.join(col) for col in res.columns.to_flat_index()]
res.head()

CANCELLED_sum  CANCELLED_mean  CANCELLED_size  \
ORG_AIR DEST_AIR                                                  
ATL     ABE                   0             0.0              31   
        ABQ                   0             0.0              16   
        ABY                   0             0.0              19   
        ACY                   0             0.0               6   
        AEX                   0             0.0              40   

                  AIR_TIME_mean  AIR_TIME_var  
ORG_AIR DEST_AIR                               
ATL     ABE           96.387097     45.778495  
        ABQ          170.500000     87.866667  
        ABY           28.578947      6.590643  
        ACY           91.333333     11.466667  
        AEX           78.725000     47.332692

In [8]:
# 열을 펼치는 함수를 만들어 pipe() 메서드 체인도 가능
def flatten_cols(df):
    df.columns = ['_'.join(col) for col in df.columns.to_flat_index()]
    return df
(flights
.groupby(['ORG_AIR', 'DEST_AIR'])
.agg({'CANCELLED':['sum', 'mean', 'size'], 'AIR_TIME':['mean', 'var']})
.pipe(flatten_cols)
.head()
)

CANCELLED_sum  CANCELLED_mean  CANCELLED_size  \
ORG_AIR DEST_AIR                                                  
ATL     ABE                   0             0.0              31   
        ABQ                   0             0.0              16   
        ABY                   0             0.0              19   
        ACY                   0             0.0               6   
        AEX                   0             0.0              40   

                  AIR_TIME_mean  AIR_TIME_var  
ORG_AIR DEST_AIR                               
ATL     ABE           96.387097     45.778495  
        ABQ          170.500000     87.866667  
        ABY           28.578947      6.590643  
        ACY           91.333333     11.466667  
        AEX           78.725000     47.332692

In [9]:
(flights
# 그룹화열 중 하나가 'category' 형식일 경우 그냥 groupby를 수행하면 카티션 곱 발생
.assign(ORG_AIR=flights.ORG_AIR.astype('category'))
# groupby() 메서드에 observed=True 인자 전달하여 카티션 곱 제거
.groupby(['ORG_AIR', 'DEST_AIR'], observed=True)
.agg({'CANCELLED':['sum', 'mean', 'size'], 'AIR_TIME':['mean', 'var']})
.head()
)

CANCELLED                   AIR_TIME            
                       sum      mean size        mean         var
ORG_AIR DEST_AIR                                                 
LAX     ABQ              1  0.018182   55   89.259259   29.403215
        ANC              0  0.000000    7  307.428571   78.952381
        ASE              1  0.038462   26  102.920000  102.243333
        ATL              0  0.000000  174  224.201149  127.155837
        AUS              0  0.000000   80  150.537500   57.897310

In [10]:
airline_info = (flights
.groupby(['AIRLINE', 'WEEKDAY'])
# 집계열이 최상위 level, 집계함수가 하위 level로 작동
.agg({'DIST':['sum', 'mean'], 'ARR_DELAY':['min', 'max']})
.astype('int')
)
airline_info.head()

DIST       ARR_DELAY     
                     sum  mean       min  max
AIRLINE WEEKDAY                              
AA      1        1455386  1139       -60  551
        2        1358256  1107       -52  725
        3        1496665  1117       -45  473
        4        1452394  1089       -46  349
        5        1427749  1122       -41  732

In [11]:
# columns 속성의 get_level_values() 메서드 활용하여 희망하는 열 출력
airline_info.columns.get_level_values(0), airline_info.columns.get_level_values(1)

(Index(['DIST', 'DIST', 'ARR_DELAY', 'ARR_DELAY'], dtype='object'),
 Index(['sum', 'mean', 'min', 'max'], dtype='object'))

In [12]:
# 기본적으로 groupby 작업은 모든 그룹화 열을 인덱스로
(flights
# as_index=False 인자 전달하여 인덱스 미지정 가능(groupby 작업 후 reset_index()를 체인시키는 것과 같은 결과)
# sort 파라미터도 활용 가능하며, 기본값은 True -> 데이터가 나타나는 순서와 동일하게 유지하려면 sort=False 인자 전달
.groupby(['AIRLINE'], as_index=False)['DIST']
.agg('mean')
.round(0)
.head()
)

,AIRLINE,DIST
0,AA,1114.0
1,AS,1066.0
2,B6,1772.0
3,DL,866.0
4,EV,460.0


In [13]:
college = pd.read_csv(p1 / 'college.csv')

In [14]:
# pandas나 numpy에 없는 사용자 정의 함수를 통해 집계 가능
# 한 기관의 평균에서의 최대 표준편차를 구하는 함수 작성
def max_deviation(ser):
    std_score = (ser - ser.mean()) / ser.std()
    # Series에 max() 메서드를 연결시켜 단일 스칼라 값 반환
    return std_score.abs().max()
# pandas는 기본적으로 함수 이름을 반환된 열의 이름으로 사용 -> __name__ 함수 속성을 사용해 수정 가능
max_deviation.__name__ = 'Max Deviation'
(college
.groupby('STABBR')['UGDS']
# 내장된 함수와 사용자 정의 함수 함께 사용 가능
.agg([max_deviation, 'count'])
.round(1)
.head()
)

,Max Deviation,count
STABBR,,
AK,2.6,10
AL,5.8,89
AR,6.3,82
AS,NaN,1
AZ,9.9,126


In [15]:
# groupby의 agg() 메서드는 agg(func, *args, **kwargs)로 구성 -> *args, **kwargs 파라미터 사용해 인자를 함수에 전달 가능
def pct_between(ser, low=1000, high=3000):
    # low이상 high이하인 경우 True, 아니면 False -> mead() 메서드를 통해 평균을 내면서 단일 스칼라 반환
    # between() 메서드는 Series에서만 가능
    return ser.between(low, high).mean() * 100
(college
.groupby(['STABBR', 'RELAFFIL'])['UGDS']
# agg() 메서드에 함수 및 *args 전달(함수들을 전달할 때는 리스트를 활용하지만 *args, **kwargs 전달 시에는 리스트 미사용)
.agg(pct_between, 1000, 10000)
.round(1)
.head()
)

STABBR  RELAFFIL
AK      0           42.9
        1            0.0
AL      0           45.8
        1           37.5
AR      0           39.7
Name: UGDS, dtype: float64

In [16]:
# 사용자 정의 함수의 파라미터를 사용하여 이름을 지정하고 싶은 경우, 함수 내에서 __name__ 속성 사용 필요
def between_n_m(n, m):
    def wrapper(ser):
        return pct_between(ser, n, m)
    wrapper.__name__ = f'between_{n}_{m}'
    return wrapper

In [17]:
# groupby 객체 검사 -> groupby() 메서드를 사용한 결과는 groupby 객체
grouped = college.groupby(['STABBR', 'RELAFFIL'])
# ngroups 속성을 통해 그룹 개수 확인 가능
type(grouped), grouped.ngroups

(pandas.core.groupby.generic.DataFrameGroupBy, 112)

In [18]:
# groups 속성을 통해 그룹화열이 키, 원본 데이터의 인덱스가 값인 딕셔너리 반환 가능
# get_group() 메서드에 키를 전달해 해당 그룹 DataFrame 반환 가능
grouped.groups[('AK', 0)], grouped.get_group(('AK', 0)).head()

(Int64Index([60, 62, 63, 65, 66, 67, 5171], dtype='int64'),
                                     INSTNM       CITY STABBR  HBCU  MENONLY  \
 60          University of Alaska Anchorage  Anchorage     AK   0.0      0.0   
 62          University of Alaska Fairbanks  Fairbanks     AK   0.0      0.0   
 63          University of Alaska Southeast     Juneau     AK   0.0      0.0   
 65  AVTEC-Alaska's Institute of Technology     Seward     AK   0.0      0.0   
 66               Charter College-Anchorage  Anchorage     AK   0.0      0.0   
 
     WOMENONLY  RELAFFIL  SATVRMID  SATMTMID  DISTANCEONLY  ...  UGDS_2MOR  \
 60        0.0         0       NaN       NaN           0.0  ...     0.0980   
 62        0.0         0       NaN       NaN           0.0  ...     0.0401   
 63        0.0         0       NaN       NaN           0.0  ...     0.0686   
 65        0.0         0       NaN       NaN           0.0  ...     0.0529   
 66        0.0         0       NaN       NaN           0.0  ...     

In [19]:
# groupby 객체에 head() 메서드 호출하여 각 그룹 첫번째 행을 받기 가능 -> 단 head() 메서드 호출 시 인덱스 미설정
# groupby 객체에 nth() 메서드 호출 및 원하는 행 전달하여 행 받기도 가능 -> 단 nth() 메서드의 경우 인덱스가 그룹화열로 자동 설정
grouped.head(1)[['INSTNM', 'CITY']].head(), grouped.nth([0])[['INSTNM', 'CITY']].head()

(                            INSTNM        CITY
 0         Alabama A & M University      Normal
 2               Amridge University  Montgomery
 43      Prince Institute-Southeast    Elmhurst
 60  University of Alaska Anchorage   Anchorage
 61            Alaska Bible College      Palmer,
                                                 INSTNM         CITY
 STABBR RELAFFIL                                                    
 AK     0                University of Alaska Anchorage    Anchorage
        1                          Alaska Bible College       Palmer
 AL     0                      Alabama A & M University       Normal
        1                            Amridge University   Montgomery
 AR     0         University of Arkansas at Little Rock  Little Rock)

In [20]:
# groupby 메서드 시 집계 대신 그룹 필터링 가능 -> filter() 메서드 활용(filter() 메서드 적용 시 원래 인덱스 그대로 유지, 값이 아닌 열을 필터링)
# DataFrame의 filter() 메서드와는 별개
college = pd.read_csv(p1 / 'college.csv', index_col='INSTNM')
grouped = college.groupby('STABBR')
# groupby 객체의 ngroups 속성을 통해 그룹 수 확인
grouped.ngroups, college['STABBR'].nunique()

(59, 59)

In [21]:
# groupby의 filter() 메서드 활용 시 사용자 정의 함수 -> 현재 그룹의 DataFrame을 취하고 불리언을 반환해야
def check_minority(df, threshold=0.5):
    minority_pct = 1 - df['UGDS_WHITE']
    # 학생 수에서 minority_pct를 곱한 후(이러면 Series로 반환) sum() 메서드 체인(이러면 단일 스칼라 반환)
    total_minority = (df['UGDS'] * minority_pct).sum()
    # 학생 수의 합을 단일 스칼라 반환
    total_ugds = df['UGDS'].sum()
    total_minority_pct = total_minority / total_ugds
    # 하나의 불리언 반환 -> 해당 그룹이 True인지 False인지를 확인 가능
    return total_minority_pct > threshold

In [22]:
# groupby 객체 filter() 메서드에 함수 및 *args 전달
college_filtered = grouped.filter(check_minority, 0.5)
college_filtered.head()

,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
INSTNM,,,,,,,,,,,,,,,,,,,,,
Everest College-Phoenix,Phoenix,AZ,0.0,0.0,0.0,1,NaN,NaN,0.0,4102.0,...,0.0373,0.0,0.1026,0.4749,0,0.8291,0.7151,0.6700,28600,9500
Collins College,Phoenix,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,83.0,...,0.0241,0.0,0.3855,0.3373,0,0.7205,0.8228,0.4764,25700,47000
Empire Beauty School-Paradise Valley,Phoenix,AZ,0.0,0.0,0.0,1,NaN,NaN,0.0,25.0,...,0.0400,0.0,0.0000,0.1600,0,0.6349,0.5873,0.4651,17800,9588
Empire Beauty School-Tucson,Tucson,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,126.0,...,0.0000,0.0,0.0079,0.2222,1,0.7962,0.6615,0.4229,18200,9833
Thunderbird School of Global Management,Glendale,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,1.0,...,0.0000,0.0,0.0000,1.0000,0,0.0000,0.0000,0.0000,118900,PrivacySuppressed


In [23]:
# 기존 college DataFrame과의 차이 확인
college.shape, college_filtered.shape, college_filtered.groupby('STABBR').ngroups

((7535, 26), (3028, 26), 20)

In [24]:
# groupby 객체의 transform() 메서드 -> 원래 인덱스를 그대로 유지하면서 결과 출력, 집계나 필터링은 미수행
weight_loss = pd.read_csv(p1 / 'weight_loss.csv')

In [25]:
def pct_loss(ser):
    # Series의 각 객체를 Series.iloc[0]과 비교하여 변화율 확인 -> Series 반환
    # transform() 메서드 적용 시 그룹을 월별로 지정해서 수행 가능
    return ((ser - ser.iloc[0]) / ser.iloc[0]) * 100

In [26]:
(weight_loss
# assign() 메서드 활용하여 새 열 생성하면서 groupby() 객체의 transform() 메서드 활용
.assign(percent_loss=weight_loss.groupby(['Name', 'Month'])['Weight'].transform(pct_loss).round(1))
# query() 메서드를 통해 Week 4만 추출하여 승자 비교
.query("Week == 'Week 4'")
# pivot() 메서드 이용, 월을 인덱스, 이름을 열, 감량수치를 값으로 지정하여 집계된 데이터 생성
.pivot(index='Month', columns='Name', values='percent_loss')
# reindex() 메서드 이용하여 인덱스 재정렬
.reindex(['Jan', 'Feb', 'Mar', 'Apr'])
# assign() 메서드 활용하여 새 열 생성 -> numpy의 where() 함수 활용하여 승자 출력
.assign(Winner=lambda df: np.where(df['Amy'] > df['Bob'], 'Bob', 'Amy'))
)

Name,Amy,Bob,Winner
Month,,,
Jan,-3.6,-2.7,Amy
Feb,-8.9,-5.3,Amy
Mar,-1.7,-2.6,Bob
Apr,-5.3,-4.2,Amy


In [28]:
# groupby 객체의 각 함수는 반환해야 하는 특정 출력 존재 -> agg()는 스칼라, filter()는 불리언, transform()은 길이가 같은 Series/DataFrame
# but apply() 메서드는 매우 유연
college = pd.read_csv(p1 / 'college.csv')
subset = ['UGDS', 'SATMTMID', 'SATVRMID']
# subset 파라미터 활용하여 찾을 열을 제한 가능
college2 = college.dropna(subset=subset)

In [29]:
# 주별 수학점수 가중평균 계산을 위한 함수
def weighted_math_average(df):
    # 학교별 수학점수 * 학생 수를 Series로 출력
    weighted_math = df['UGDS'] * df['SATMTMID']
    # 수학점수 * 학생수를 학생수로 나눠 가중평균 구하기
    return int(weighted_math.sum() / df['UGDS'].sum())
# groupby 객체의 apply() 메서드 활용하여 그룹별로 새로운 값 추출
# apply() 메서드의 경우 별도의 집계열이 필요가 없음 -> 여러 개의 열에 대한 연산을 완료하는데 있어 최적
college2.groupby('STABBR').apply(weighted_math_average).head()

STABBR
AK    503
AL    536
AR    529
AZ    569
CA    564
dtype: int64

In [30]:
# apply() 메서드는 함수의 Series를 반환할 경우 다수의 새로운 열 생성 가능 -> Series의 index가 반환 DataFrame의 열로 변환
def weighted_average(df):
   weight_m = df['UGDS'] * df['SATMTMID']
   weight_v = df['UGDS'] * df['SATVRMID']
   wm_avg = weight_m.sum() / df['UGDS'].sum()
   wv_avg = weight_v.sum() / df['UGDS'].sum()
   data = {'w_math_avg': wm_avg, 'w_verbal_avg': wv_avg,
           'math_avg': df['SATMTMID'].mean(), 'verbal_avg': df['SATVRMID'].mean(),
           'count': len(df)}
   # 함수의 반환 값을 Series로 설정
   return pd.Series(data)
(college2
.groupby('STABBR')
.apply(weighted_average)
.astype(int)
.head()
)

,w_math_avg,w_verbal_avg,math_avg,verbal_avg,count
STABBR,,,,,
AK,503,555,503,555,1
AL,536,533,504,508,21
AR,529,504,515,491,16
AZ,569,557,536,538,6
CA,564,539,562,549,72


In [35]:
def calculate_means(df):
    # df_means라는 새로운 DataFrame 생성 -> index 이름 설정
    df_means = pd.DataFrame(index=['Arithmetic',' Weighted', 'Geometric', 'Harmonic'])
    # 'SATMTMID', 'SATVRMID' 열 순환
    cols = ['SATMTMID', 'SATVRMID']
    for col in cols:
        # 각 열 Series의 산술평균 값 구하기
        arithmetic = df[col].mean()
        # 각 열 Series의 가중평균 값 구하기
        weighted = np.average(df[col], weights=df['UGDS'])
        # 각 열 Series의 기하평균 값 구하기
        geometric = ss.gmean(df[col])
        # 각 열 Series의 조화평균 값 구하기
        harmonic = ss.hmean(df[col])
        # 새로 만들어진 df_means DataFrame에서 각 열을 df_means.index에 맞게 값 매치
        df_means[col] = [arithmetic, weighted, geometric, harmonic]
    # df_means DataFrame에 'count'열 추가
    df_means['count'] = len(df)
    # DataFrame 반환 -> 인덱스(산술, 가중, 기하, 조화평균), 열(SATMTMID, SATVRMID, count)
    return df_means.astype('int')
(college2
.groupby('STABBR')
# apply() 메서드에 출력이 DataFrame인 함수 전달 -> 그룹 내 DataFrame 형식으로 출력
.apply(calculate_means)
.head()
)

SATMTMID  SATVRMID  count
STABBR                                      
AK     Arithmetic       503       555      1
        Weighted        503       555      1
       Geometric        503       555      1
       Harmonic         503       555      1
AL     Arithmetic       504       508     21